In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import xesmf as xe
import pandas as pd
from flaml import AutoML
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.manifold import LocallyLinearEmbedding
from scipy import stats
import openturns as ot
from scipy.stats import genextreme as gev
from scipy.stats import genpareto as gpd

In [ ]:
#ctf_ensemble = np.loadtxt("ctf_ensemble_preds.txt")
#fact_ensemble = np.loadtxt("fact_ensemble_preds.txt")
#ctf_ensemble = np.loadtxt("ctf_ensemble_preds_v2.txt")
#fact_ensemble = np.loadtxt("fact_ensemble_preds_v2.txt")
#ctf_ensemble = np.loadtxt("ctf_ensemble_preds_v3.txt")
#fact_ensemble = np.loadtxt("fact_ensemble_preds_v2.txt")
#ctf_ensemble = np.loadtxt("ctf_ensemble_preds_v4.txt")
#fact_ensemble = np.loadtxt("fact_ensemble_preds_v2.txt")
ctf_ensemble = np.loadtxt("ctf_ensemble_preds_v4.txt")
fact_ensemble = np.loadtxt("fact_ensemble_preds_v2.txt")

In [ ]:
density_ctf = stats.gaussian_kde(ctf_ensemble)
density_fact = stats.gaussian_kde(fact_ensemble)

In [ ]:
X_lim = np.linspace(0,300)
y_ctf = density_ctf(X_lim)
y_fact = density_fact(X_lim)

In [ ]:
plt.title("KDE: Counterfactual and Factual Ensemble ACE Predictions")
plt.xlabel("Predicted 2020 Atlantic Seasonal ACE")
plt.plot(X_lim,y_ctf,label="Counterfactual")
plt.plot(X_lim,y_fact, label="Factual")
plt.legend()
plt.show()

In [ ]:
ace_2020 = 180.3725

In [ ]:
p0 = 1 - density_ctf.integrate_box_1d(0,ace_2020)
p1 = 1 - density_fact.integrate_box_1d(0,ace_2020)
p2 = 1 - density_fact.integrate_box_1d(0,140)
print(p0*100)
print(p1*100)
print(p2*100)

In [ ]:
pn = 1-p0/p1
ps = 1 - (1-p1)/(1-p0)
pns = p1-p0
rr = p1/p0

In [ ]:
print('Necessity: ' + str(pn))
print('Sufficiency: ' + str(ps))
print('Necessity & Sufficiency: ' + str(pns))
print('Risk ratio: ' + str(rr))

In [ ]:
def gen_trip(ace):
    p0 = 1 - density_ctf.integrate_box_1d(0,ace)
    p1 = 1 - density_fact.integrate_box_1d(0,ace)
    pn = 1-p0/p1
    ps = 1 - (1-p1)/(1-p0)
    pns = p1-p0
    return pn,ps,pns

In [ ]:
to_check = np.linspace(30,250)

In [ ]:
pn_l = []
ps_l = []
pns_l = []
for val in to_check:
    pn,ps,pns = gen_trip(val)
    pn_l.append(pn)
    ps_l.append(ps)
    pns_l.append(pns)

In [ ]:



plt.plot(to_check,pn_l,label="PN")
plt.plot(to_check,ps_l, label="PS")
plt.plot(to_check,pns_l, label="PNS")
plt.legend()
plt.show()

In [ ]:
def gen_trip_time(ace,t):
    p0 = 1 - density_ctf.integrate_box_1d(0,ace)
    p1 = 1 - density_fact.integrate_box_1d(0,ace)
    p0 = 1 - (1-p0)**t
    p1 = 1 - (1-p1)**t
    pn = 1-p0/p1
    ps = 1 - (1-p1)/(1-p0)
    pns = p1-p0
    return pn,ps,pns

In [ ]:
times_before = np.linspace(1,200)
pn_lt = []
ps_lt = []
pns_lt = []
for t in times_before:
    pn,ps,pns = gen_trip_time(ace_2020,t)
    pn_lt.append(pn)
    ps_lt.append(ps)
    pns_lt.append(pns)

In [ ]:
plt.title("Necessity, Sufficicency, and Both Occuring by 2020")
plt.xlabel("Observational Period")
plt.plot(times_before,pn_lt,label="PN")
plt.plot(times_before,ps_lt, label="PS")
plt.plot(times_before,pns_lt, label="PNS")
plt.legend()
plt.show()

In [ ]:
shp_gev,loc_gev,sc_gev = gev.fit(ctf_ensemble)
fshp_gev,floc_gev,fsc_gev = gev.fit(fact_ensemble)

In [ ]:
shp_gpd,loc_gpd,sc_gpd =  gpd.fit(ctf_ensemble)
fshp_gpd,floc_gpd,fsc_gpd = gpd.fit(fact_ensemble)

In [ ]:
l = loc_gev + sc_gev / shp_gev
xx = np.linspace(60,250)
yy = gev.pdf(xx, shp_gev, loc_gev , sc_gev)
plt.plot(xx, yy, label="Counterfactual")
fl = floc_gev + fsc_gev / fshp_gev
#fxx = np.linspace(60,250, num=71)
fyy = gev.pdf(xx, fshp_gev, floc_gev , fsc_gev)
plt.plot(xx, fyy, label="Factual")
plt.title("GEV: Counterfactual and Factual Ensemble ACE Predictions")
plt.xlabel("Predicted 2020 Atlantic Seasonal ACE")
plt.legend()
plt.show()

In [ ]:
def gev_gen_trip(ace):
    p0 = 1 - gev.cdf(ace, shp_gev, loc_gev , sc_gev)
    p1 = 1 - gev.cdf(ace, fshp_gev, floc_gev , fsc_gev)
    pn = 1-p0/p1
    ps = 1 - (1-p1)/(1-p0)
    pns = p1-p0
    rr = p1/p0
    return pn,ps,pns,rr
def time_gev_gen_trip(ace,t):
    p0 = 1 - gev.cdf(ace, shp_gev, loc_gev , sc_gev)
    p1 = 1 - gev.cdf(ace, fshp_gev, floc_gev , fsc_gev)
    p0 = 1 - (1-p0)**t
    p1 = 1 - (1-p1)**t
    pn = 1-p0/p1
    ps = 1 - (1-p1)/(1-p0)
    pns = p1-p0
    rr = p1/p0
    return pn,ps,pns,rr

In [ ]:
pn,ps,pns,rr = gev_gen_trip(ace_2020)
print('Necessity: ' + str(pn))
print('Sufficiency: ' + str(ps))
print('Necessity & Sufficiency: ' + str(pns))
print('Risk Ratio: ' + str(rr))

In [ ]:
to_check = np.linspace(30,250)
pn_l = []
ps_l = []
pns_l = []
for val in to_check:
    pn,ps,pns = gev_gen_trip(val)
    pn_l.append(pn)
    ps_l.append(ps)
    pns_l.append(pns)
plt.title("Necessity, Sufficicency, and Both vs. Possible ACEs")
plt.xlabel("Possible 2020 Atlantic Seasonal ACE")
plt.plot(to_check,pn_l,label="PN")
plt.plot(to_check,ps_l, label="PS")
plt.plot(to_check,pns_l, label="PNS")
plt.legend()
plt.show()

In [ ]:
times_before = np.linspace(1,30)
pn_lt = []
ps_lt = []
pns_lt = []
for t in times_before:
    pn,ps,pns = time_gev_gen_trip(ace_2020,t)
    pn_lt.append(pn)
    ps_lt.append(ps)
    pns_lt.append(pns)
plt.title("Necessity, Sufficicency, and Both Occuring by 2020")
plt.xlabel("Observational Period")
plt.plot(times_before,pn_lt,label="PN")
plt.plot(times_before,ps_lt, label="PS")
plt.plot(times_before,pns_lt, label="PNS")
plt.legend()
plt.show()

In [ ]:
l = loc_gpd + sc_gpd / shp_gpd
xx = np.linspace(60,250)
yy = gpd.pdf(xx, shp_gpd, loc_gpd , sc_gpd)
plt.plot(xx, yy, label="Counterfactual")
fl = floc_gpd + fsc_gpd / fshp_gpd
#fxx = np.linspace(60,250, num=71)
fyy = gpd.pdf(xx, fshp_gpd, floc_gpd , fsc_gpd)
plt.plot(xx, fyy, label="Factual")
plt.legend()
plt.show()